### Amazon Cookbooks Topic-modeling: Create Categories and Scarpe for Pics

__Purpose:__
The purpose of this notebook is to:
1. Create classes for classification task
2. Calc average ratings per book for regresson
3. Code for scraping of book images   

In [1]:
import numpy as np
import pandas as pd
import re, string

#import contractions
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords, wordnet
from nltk.stem import WordNetLemmatizer
from langdetect import detect
from textblob import TextBlob

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA, LatentDirichletAllocation, TruncatedSVD, NMF
from sklearn.cluster import KMeans

from joblib import dump, load
import matplotlib.pyplot as plt
import matplotlib.cm as cm
%matplotlib inline

    
# --- Parser for reading in the Amazon json files (can be used for both reviews and metadata)
# --- credits folloing parse() method to Julian McAuley UCSD: http://jmcauley.ucsd.edu/data/amazon/ 
def parse(path):
    g = open(path, 'r')
    for l in g:
        yield json.loads(l)

#### __Read Cookbook Reviews and Cookbook Metadata__

In [8]:
# --- Read cookbooks tables:  Products from meta_cookbooks.json and reviews from Cookbooks.json
ckbk_meta_file = "../data/amzn/meta_Cookbooks.json"
ckbk_reviews_file = "../data/amzn/Cookbooks.json"

books_ASIN = {} #asin to idx lookup
reviewer_ID = {} #reviewerID to idx lookup
ratings_matrix =[]

book_no = 0
reviewer_no = 0

count = 0
for line in parse(ckbk_reviews_file):
    if 'asin' in line and 'overall' in line and 'reviewerID' in line:    
        if line['asin'] not in books_ASIN:
            book_no += 1
            books_ASIN[line['asin']] = book_no
            
        if line['reviewerID'] not in reviewer_ID:
            reviewer_no += 1
            reviewer_ID[line['reviewerID']] = reviewer_no
            
        string_to_append =  str(int(books_ASIN[line['asin']])) +","
        string_to_append += str(int(reviewer_ID[line['reviewerID']])) + ","
        string_to_append += str(line['overall'])
        
        ratings_matrix.append(string_to_append)
        
        count += 1
        
    #if count>20000:
    #    break
#print(ratings_matrix)
#print(f"Reviewers: {reviewer_ID}")
#print(f"Books: {books_ASIN}")

In [5]:
print(len(ratings_matrix))

output_file = "../data/amzn/ratings_matrix.csv"
with open(output_file, "w") as f:
    for i in ratings_matrix:
        f.write(i+'\n')

output_bookIDs = "../data/amzn/ratings_matrix_bookIDs.json"
open(output_bookIDs, "w").write(json.dumps(books_ASIN))

output_reviewerID = "../data/amzn/ratings_matrix_reviwerIDs.json"
open(output_reviewerID, "w").write(json.dumps(reviewer_ID))

428522


5314381

In [12]:
print(f"Ratings matrix has {len(books_ASIN)} books and {len(reviewer_ID)} reviwers")
print(f"...with total reviews of {len(ratings_matrix)}")


Ratings matrix has 15782 books and 210808 reviwers
...with total reviews of 428522
